## 定义目标函数和数据采集

In [36]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

# 定义目标函数 f(x) = x^2
def target_function(x):
    return x ** 2

# 生成训练集和测试集，归一化输入数据
def generate_data(n_samples=1000, test_size=200):
    # 生成训练数据（输入：x, 输出：f(x)）
    x_train = np.random.uniform(-5, 5, n_samples)
    y_train = target_function(x_train)

    # 归一化输入数据到 [-1, 1] 之间
    x_train = (x_train - np.min(x_train)) / (np.max(x_train) - np.min(x_train)) * 2 - 1  # [-1, 1]
    x_test = np.random.uniform(-5, 5, test_size)
    y_test = target_function(x_test)

    x_test = (x_test - np.min(x_test)) / (np.max(x_test) - np.min(x_test)) * 2 - 1  # [-1, 1]

    return (x_train, y_train), (x_test, y_test)



## 建立模型

In [37]:
class myModel:
    def __init__(self):
        '''声明模型对应的参数'''
        # 使用 He 初始化方法
        self.W1 = tf.Variable(tf.random.normal([1, 128], stddev=np.sqrt(2 / 1)))  # 第一层的权重
        self.b1 = tf.Variable(tf.zeros([128]))  # 第一层的偏置
        self.W2 = tf.Variable(tf.random.normal([128, 1], stddev=np.sqrt(2 / 128)))  # 第二层的权重
        self.b2 = tf.Variable(tf.zeros([1]))  # 第二层的偏置

    def __call__(self, x):
        '''实现模型函数体，返回未归一化的logits'''
        # 第一层 ReLU 激活
        hidden = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        # 第二层输出
        logits = tf.matmul(hidden, self.W2) + self.b2
        return logits

# 使用较小的学习率
optimizer = optimizers.Adam(learning_rate=0.0001)


## 计算 loss

In [38]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(tf.square(logits - labels))

@tf.function
def compute_accuracy(logits, labels):
    # 在回归任务中可以根据误差进行判断
    return tf.reduce_mean(tf.abs(logits - labels))  # 使用平均绝对误差作为准确度指标

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01 * g)  # 更新权重

    accuracy = compute_accuracy(logits, y)

    # 返回损失和准确度
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy


## 训练模型

In [43]:
# 获取训练数据和测试数据
train_data, test_data = generate_data()

# 创建模型实例
model = myModel()

# 转换为 TensorFlow Tensor
x_train = tf.constant(train_data[0].reshape(-1, 1), dtype=tf.float32)  # 输入 reshape 为 [N, 1]
y_train = tf.constant(train_data[1].reshape(-1, 1), dtype=tf.float32)  # 输出 reshape 为 [N, 1]

x_test = tf.constant(test_data[0].reshape(-1, 1), dtype=tf.float32)
y_test = tf.constant(test_data[1].reshape(-1, 1), dtype=tf.float32)

# 训练过程
for epoch in range(10000):
    loss, accuracy = train_one_step(model, optimizer, x_train, y_train)
    print(f'epoch {epoch}: loss {loss.numpy()}; accuracy {accuracy.numpy()}')

# 测试模型
loss, accuracy = test(model, x_test, y_test)
print(f'test loss {loss.numpy()}; test accuracy {accuracy.numpy()}')


epoch 0: loss 131.88571166992188; accuracy 8.5330171585083
epoch 1: loss 52.21128463745117; accuracy 4.9262261390686035
epoch 2: loss 24.72571563720703; accuracy 3.438365936279297
epoch 3: loss 14.962105751037598; accuracy 2.9155614376068115
epoch 4: loss 11.435310363769531; accuracy 2.736598253250122
epoch 5: loss 10.08663272857666; accuracy 2.684718132019043
epoch 6: loss 9.503084182739258; accuracy 2.6651909351348877
epoch 7: loss 9.193415641784668; accuracy 2.6489756107330322
epoch 8: loss 8.984782218933105; accuracy 2.631589889526367
epoch 9: loss 8.815414428710938; accuracy 2.6123545169830322
epoch 10: loss 8.662619590759277; accuracy 2.5911054611206055
epoch 11: loss 8.516592025756836; accuracy 2.5681874752044678
epoch 12: loss 8.374878883361816; accuracy 2.5442914962768555
epoch 13: loss 8.238638877868652; accuracy 2.520338773727417
epoch 14: loss 8.107049942016602; accuracy 2.496645450592041
epoch 15: loss 7.979520797729492; accuracy 2.473140239715576
epoch 16: loss 7.85558891